# IMPORTING LIBRARIES

In [ ]:
import json
import re
import os                    
import datetime
import sklearn
import bs4
import csv
import requests
import time
import urllib
import re
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from bs4 import BeautifulSoup as bs
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from keras.layers import Dense, BatchNormalization, Dropout, LSTM
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from keras import callbacks
from time import sleep

import selenium 
from datetime import date, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
np.random.seed(0)
pd.options.display.max_columns = None
pd.options.display.max_rows = None


In [ ]:
def make_df(url):
    url_re  = requests.get(url)
    soup =  bs(url_re.content,'html.parser')
    table = soup.find_all('table')
    raw_of_my_df = [row.text.splitlines() for row in table]
    raw_of_my_df = raw_of_my_df[:-9] 
    for i in range(len(raw_of_my_df)):
        raw_of_my_df[i] = raw_of_my_df[i][2:len(raw_of_my_df[i]):3]
    for i in range(len(raw_of_my_df)):
        c = ['.'.join(re.findall("\d+",str(raw_of_my_df[i][j].split()[:5])))for j in range(len(raw_of_my_df[i]))]
        list_of_df.append(c)
        index.append(dates[d] + c[0])
    l_index = [index[i] for i in range(len(index)) if len(index[i]) > 6]
    my_df_day = [list_of_df[i][1:] for i in range(len(list_of_df)) if len(list_of_df[i][1:]) == 19]
    last_index = [datetime.strptime(str(l_index[i]), '%Y%m%d').strftime('%Y-%m-%d') for i in range(len(l_index))]
    
    cols = ['Average temperature (°F)', 'Average humidity (%)',
           'Average dewpoint (°F)', 'Average barometer (in)',
           'Average windspeed (mph)', 'Average gustspeed (mph)',
           'Average direction (°deg)', 'Rainfall for month (in)',
           'Rainfall for year (in)', 'Maximum rain per minute',
           'Maximum temperature (°F)', 'Minimum temperature (°F)',
           'Maximum humidity (%)', 'Minimum humidity (%)', 'Maximum pressure',
           'Minimum pressure', 'Maximum windspeed (mph)',
           'Maximum gust speed (mph)', 'Maximum heat index (°F)']
    
    return pd.DataFrame(my_df_day, columns = cols, index = last_index)
    

In [ ]:
def make_rain_df(url):
    url_rain  = requests.get(url)
    soup_rain =  bs(url_rain.content,'html.parser')
    td_tags = soup_rain.find_all('td')
    for i in range(len(td_tags)):
        if(str(td_tags[i].get_text()) == "Daily Rain Totals"):
            j=i+1
            while(j < (len(td_tags))): 
                if len(str(td_tags[j].get_text())) > 21:
                    break
                result.append(str(td_tags[j].get_text()))
                j+=1
    columns = ['Daily Rain Totals']
    return pd.DataFrame(result, columns = columns)

In [ ]:
def make_date_df(year,month,day):
    temp.append( str(year)+str(month)+str(day))
    col=['d']
    return pd.DataFrame(temp,columns=col)

In [ ]:
result=[]
temp= []
list_of_df = []
index = []

rows_dates = pd.date_range(start = '1/1/2006',end = '7/1/2021',freq = 'M')
dates= [str(i)[:4] + str(i)[5:7] for i in rows_dates]
dates[0:5]

In [ ]:
#Crawling to get data of daily wearher

for d in range(len(dates)):
    url = "http://www.estesparkweather.net/archive_reports.php?date="
    url += dates[d]
    temp_df = make_df(url)
    data = pd.concat([temp_df])

In [ ]:
#Crawling to get data of daily rain

count_curr=0
new_rows=0
count_prev=0
k=0
s=0
r=0

for d in range(len(dates)):
    url_rain = "http://www.estesparkweather.net/archive_reports.php?date="
    url_rain+= dates[d]
    
    temp_df_rain = make_rain_df(url_rain)
    
    for i in range(len(temp_df_rain)):
        count_curr+=1
        
    new_rows += count_curr - count_prev
    count_prev += new_rows
    count_curr = 0
    
    temp_df_rain['Date'] = temp_df_rain['Daily Rain Totals'].str[-2:]
    temp_df_rain['Daily rain total (in)'] = temp_df_rain['Daily Rain Totals'].str[0:5]
    
    year =  dates[d][0:4]
    month = dates[d][4:6]
    
    for i in range(0,new_rows):
        my_date_df = make_date_df(year,month,temp_df_rain['Date'][k])
        df_date = pd.concat([my_date_df])
        k+=1
    new_rows = 0
    data_rain = pd.concat([ temp_df_rain])

In [ ]:
date

In [ ]:
data_rain

In [ ]:
data['Total Rain (in)'] = 0
data

In [ ]:
data.info()

In [ ]:
df_date['d'] = [x.replace(' ','0') for x in df_date['d']] 
df_date

In [ ]:
p=0
for d in df_date['d']:
    df_date.loc[p,"Date"] = datetime.strptime(str(d), '%Y%m%d').strftime('%Y-%m-%d')
    p+=1
df_date

In [ ]:
data['Total Rain (in)'] = data['Total Rain (in)'].astype(float)
data


In [ ]:
data_rain['Daily rain total (in)'] = data_rain['Daily rain total (in)'].astype(float)
data_rain

In [ ]:
curr_rain=0
curr_df=0
count=0
temp_rain = data.copy()
for da in df_date['Date']:
    for ind in temp_rain.index:
        if(ind == da):
            count+=1
            temp_rain['Total Rain (in)'][curr_df]=data_rain.loc[curr_rain,'Daily rain total (in)']
            curr_df+=1
            curr_df=0
            break
        else:
            curr_df+=1 
    curr_df=0
    curr_rain+=1

In [ ]:
temp_rain

In [ ]:
#mydf.rename(columns={'first name':'First_Name'},inplace=True)
#data.rename(columns={'Unnamed: 0':'Date'},inplace=True)
#data

In [ ]:
temp = temp_rain.copy()

In [ ]:
num=0
temp['Total Rain (mm)'] = 0
temp['Total Rain (mm)'] = temp['Total Rain (mm)'].astype(float)
for i in range(len(temp['Total Rain (in)'])):
    temp['Total Rain (mm)'][i] += float(temp['Total Rain (in)'][i] * 25.4)
temp

In [ ]:
dates_range= pd.date_range(start = '1/1/2006',end = '7/1/2021',freq = 'D')
dates_range

In [ ]:
 col = ['Date', 'Average temperature (°F)', 'Average humidity (%)',
       'Average dewpoint (°F)', 'Average barometer (in)',
       'Average windspeed (mph)', 'Average gustspeed (mph)',
       'Average direction (°deg)', 'Rainfall for month (in)',
       'Rainfall for year (in)', 'Maximum rain per minute',
       'Maximum temperature (°F)', 'Minimum temperature (°F)',
       'Maximum humidity (%)', 'Minimum humidity (%)', 'Maximum pressure',
       'Minimum pressure', 'Maximum windspeed (mph)',
       'Maximum gust speed (mph)', 'Maximum heat index (°F)',
       'Total Rain (in)', 'Total Rain (mm)']
df_total = pd.DataFrame(columns=col)
df_total['Date'] = pd.to_datetime(dates_range)
df_total

In [ ]:
col_inx=[]
for co in temp.columns:
    col_inx.append(co)
col_inx=col_inx[1:]
col_inx

In [ ]:
temp.to_csv("My_data_CO_w.csv")

In [ ]:
my_data=pd.read_csv("My_data_CO_w.csv")
my_data

In [ ]:
my_data.rename(columns={'Unnamed: 0':'Date'},inplace=True)
my_data

In [ ]:
curr_df=0
curr_rain=0
count=0
coun=0
for ind in my_data['Date']:
    for da in df_total['Date']:
        count+=1
        if(ind == (str(da)[:10])):
            coun+=1
            for co in col_inx:
                df_total.loc[curr_df,co] = my_data.loc[curr_rain,co]
            curr_df+=1
            curr_df=0
            break
        else:
             curr_df+=1 
    curr_df=0
    curr_rain+=1
df_total

In [ ]:
df_total.to_csv("DataSet.csv")